In [0]:
%sql
CREATE TABLE workspace.default.change_log AS
WITH yesterday AS (
    SELECT *, 'yesterday' AS source FROM workspace.default.gp_latest_20250809_233228s
),
today AS (
    SELECT *, 'today' AS source FROM workspace.default.gp_latest_20250810_221255s
)
SELECT
    COALESCE(y.OBJECT_ID, t.OBJECT_ID) AS satellite_id,
    COALESCE(y.EPOCH, t.EPOCH) AS epoch,
    CASE
        WHEN y.source IS NULL THEN 'new_record'
        WHEN t.source IS NULL THEN 'deleted_record'
        WHEN y.TLE_LINE1 <> t.TLE_LINE1 OR y.TLE_LINE2 <> t.TLE_LINE2 THEN 'modified_record'
        ELSE 'no_change'
    END AS change_type
FROM yesterday y
FULL OUTER JOIN today t
ON y.OBJECT_ID = t.OBJECT_ID AND y.EPOCH = t.EPOCH;



MERGE INTO workspace.default.change_log AS target
USING (
    WITH yesterday AS (
        SELECT *, 'yesterday' AS source FROM workspace.default.gp_latest_20250809_233228s
    ),
    today AS (
        SELECT *, 'today' AS source FROM workspace.default.gp_latest_20250812_033548s
    )
    SELECT
        COALESCE(y.OBJECT_ID, t.OBJECT_ID) AS satellite_id,
        COALESCE(y.EPOCH, t.EPOCH) AS epoch,
        CASE
            WHEN y.source IS NULL THEN 'new_record'
            WHEN t.source IS NULL THEN 'deleted_record'
            WHEN y.TLE_LINE1 <> t.TLE_LINE1 OR y.TLE_LINE2 <> t.TLE_LINE2 THEN 'modified_record'
            ELSE 'no_change'
        END AS change_type
    FROM yesterday y
    FULL OUTER JOIN today t
    ON y.OBJECT_ID = t.OBJECT_ID AND y.EPOCH = t.EPOCH
) AS source
ON target.satellite_id = source.satellite_id AND target.epoch = source.epoch
WHEN MATCHED AND source.change_type <> 'no_change' THEN
    UPDATE SET target.change_type = source.change_type
WHEN NOT MATCHED THEN
    INSERT (satellite_id, epoch, change_type)
    VALUES (source.satellite_id, source.epoch, source.change_type);





